# [Exploration 08] Movielens 영화 추천 

## 프로젝트 설명 
- 추천시스템 실습
- Movielens 데이터셋을 사용    
`
$ wget http://files.grouplens.org/datasets/movielens/ml-1m.zip
`
    - Movielens : 영화에 대한 평점을 매긴 데이터
    - 평점 데이터는 explicit데이터이지만, implicit데이터로 간주하고 테스트 진행 
    - 3점 미만은 비선호로 간주해 데이터셋에서 제외
- MF모델 사용

## 프로젝트 목표
-

## 01. load and preprocess data
- column name을 생성합니다.

In [1]:
import os
import pandas as pd

rating_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/ratings.dat'
ratings_cols = ['user_id', 'movie_id', 'rating', 'timestamp']      # column name이 없기 때문에 생성
ratings = pd.read_csv(rating_file_path, sep='::', names=ratings_cols, engine='python')
print(ratings.shape)
ratings.head()

(1000209, 4)


,user_id,movie_id,rating,timestamp
0,1,1193,5,978300760
1,1,661,3,978302109
2,1,914,3,978301968
3,1,3408,4,978300275
4,1,2355,5,978824291


**메타 데이터('movies.dat')으로 영화 제목을 확인합니다.**

In [2]:
movie_file_path=os.getenv('HOME') + '/aiffel/recommendata_iu/data/ml-1m/movies.dat'
cols = ['movie_id', 'title', 'genre'] 
movies = pd.read_csv(movie_file_path, sep='::', names=cols, engine='python', encoding = 'ISO-8859-1')
print(movies.shape)
movies.head()

(3883, 3)


,movie_id,title,genre
0,1,Toy Story (1995),Animation|Children's|Comedy
1,2,Jumanji (1995),Adventure|Children's|Fantasy
2,3,Grumpier Old Men (1995),Comedy|Romance
3,4,Waiting to Exhale (1995),Comedy|Drama
4,5,Father of the Bride Part II (1995),Comedy


**ratings, movies의 데이터 갯수가 같은지 확인합니다.**

In [3]:
print(ratings['movie_id'].nunique())
print(movies['movie_id'].nunique())

3706
3883


- **데이터를 다루기 쉽도록 movies와 ratings를 합쳐주도록 하겠습니다.**

In [4]:
ratings = pd.merge(ratings,movies, how='outer',on='movie_id') 
orginal_data_size = len(ratings)

In [5]:
ratings

,user_id,movie_id,rating,timestamp,title,genre
0,1.0,1193,5.0,978300760.0,One Flew Over the Cuckoo's Nest (1975),Drama
1,2.0,1193,5.0,978298413.0,One Flew Over the Cuckoo's Nest (1975),Drama
2,12.0,1193,4.0,978220179.0,One Flew Over the Cuckoo's Nest (1975),Drama
3,15.0,1193,4.0,978199279.0,One Flew Over the Cuckoo's Nest (1975),Drama
4,17.0,1193,5.0,978158471.0,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
1000381,NaN,3650,NaN,NaN,Anguish (Angustia) (1986),Horror
1000382,NaN,3750,NaN,NaN,Boricua's Bond (2000),Drama
1000383,NaN,3829,NaN,NaN,Mad About Mambo (2000),Comedy|Romance
1000384,NaN,3856,NaN,NaN,Autumn Heart (1999),Drama


- 정수로 된 user_id, movie_id, 평점(rating), 재생시간(timestamp)을 확인    
- 총 데이터 100만여개 확인

**3점 이상의 데이터만 남깁니다.**
- f string 소수점 코드 :.2%

In [6]:
ratings = ratings[ratings['rating']>=3]
filtered_data_size = len(ratings)

print(f'orginal_data_size: {orginal_data_size}, filtered_data_size: {filtered_data_size}')
print(f'Ratio of Remaining Data is {filtered_data_size / orginal_data_size:.2%}')

orginal_data_size: 1000386, filtered_data_size: 836478
Ratio of Remaining Data is 83.62%


**파일이름(ratings)와 겹치지 않도록 컬럼 *rating*의 이름을 *count*로 바꿉니다.**

In [7]:
ratings.rename(columns={'rating':'count'}, inplace=True)
ratings['count']

/home/aiffel-dj26/anaconda3/envs/aiffel/lib/python3.7/site-packages/pandas/core/frame.py:4446: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  errors=errors,


0          5.0
1          5.0
2          4.0
3          4.0
4          5.0
          ... 
1000203    3.0
1000204    5.0
1000205    3.0
1000207    5.0
1000208    4.0
Name: count, Length: 836478, dtype: float64

In [8]:
movies['movie_id'].nunique()

3883

In [9]:
ratings['movie_id'].nunique()

3628

**ratings 데이터로 학습한 후, 메타 데이터로 예측한 영화의 제목, 장르를 확인해 모델을 평가할 수 있을 것으로 보입니다.**

- id, title을 확인하는 함수 생성

In [11]:
def id_to_title(id):
    return movies['title'][movies['movie_id']==id].values[0]

In [12]:
id_to_title(2)

'Jumanji (1995)'

In [13]:
def title_to_id(title):
    return movies['movie_id'][movies['title']==title].values[0].item()

In [14]:
title_to_id('Jumanji (1995)')

2

## 02. 데이터 파헤치기 
- 더 세부적인 정보를 얻기 위해 다음을 출력해보겠습니다.
    - ratings에 있는 유니크한 영화 개수
    - rating에 있는 유니크한 사용자 수
    - 가장 인기 있는 영화 30개(인기순)  

In [16]:
ratings.head()

,user_id,movie_id,count,timestamp,title,genre
0,1.0,1193,5.0,978300760.0,One Flew Over the Cuckoo's Nest (1975),Drama
1,2.0,1193,5.0,978298413.0,One Flew Over the Cuckoo's Nest (1975),Drama
2,12.0,1193,4.0,978220179.0,One Flew Over the Cuckoo's Nest (1975),Drama
3,15.0,1193,4.0,978199279.0,One Flew Over the Cuckoo's Nest (1975),Drama
4,17.0,1193,5.0,978158471.0,One Flew Over the Cuckoo's Nest (1975),Drama


In [17]:
# 유니크한 사용자 수 
ratings['user_id'].nunique()

6039

In [18]:
# 유니크한 영화 개수 
ratings['movie_id'].nunique()

3628

In [19]:
# 인기 많은 영화
movies_count = ratings.groupby('movie_id')['user_id'].count()   # 영화별 grouping, 사용자 수를 카운트
movies_count.sort_values(ascending=False).head(30)              # 상위 30편

movie_id
2858    3211
260     2910
1196    2885
1210    2716
2028    2561
589     2509
593     2498
1198    2473
1270    2460
2571    2434
480     2413
2762    2385
608     2371
110     2314
1580    2297
527     2257
1197    2252
2396    2213
1617    2210
318     2194
858     2167
1265    2121
1097    2102
2997    2066
2716    2051
296     2030
356     2022
1240    2019
1       2000
457     1941
Name: user_id, dtype: int64

**id로 되어있는 영화들, 영화의 제목을 알아봅시다!**

In [20]:
movies_30 = movies_count.sort_values(ascending=False).head(30)
movies_30.keys()

Int64Index([2858,  260, 1196, 1210, 2028,  589,  593, 1198, 1270, 2571,  480,
            2762,  608,  110, 1580,  527, 1197, 2396, 1617,  318,  858, 1265,
            1097, 2997, 2716,  296,  356, 1240,    1,  457],
           dtype='int64', name='movie_id')

- 순위와 함꼐 뽑기

In [21]:
for idx, i in zip(movies_30.index, movies_30.keys()):
    print(idx, id_to_title(i))

2858 American Beauty (1999)
260 Star Wars: Episode IV - A New Hope (1977)
1196 Star Wars: Episode V - The Empire Strikes Back (1980)
1210 Star Wars: Episode VI - Return of the Jedi (1983)
2028 Saving Private Ryan (1998)
589 Terminator 2: Judgment Day (1991)
593 Silence of the Lambs, The (1991)
1198 Raiders of the Lost Ark (1981)
1270 Back to the Future (1985)
2571 Matrix, The (1999)
480 Jurassic Park (1993)
2762 Sixth Sense, The (1999)
608 Fargo (1996)
110 Braveheart (1995)
1580 Men in Black (1997)
527 Schindler's List (1993)
1197 Princess Bride, The (1987)
2396 Shakespeare in Love (1998)
1617 L.A. Confidential (1997)
318 Shawshank Redemption, The (1994)
858 Godfather, The (1972)
1265 Groundhog Day (1993)
1097 E.T. the Extra-Terrestrial (1982)
2997 Being John Malkovich (1999)
2716 Ghostbusters (1984)
296 Pulp Fiction (1994)
356 Forrest Gump (1994)
1240 Terminator, The (1984)
1 Toy Story (1995)
457 Fugitive, The (1993)


**영화와 유저에 대한 통계, 영화 재생 시간에 대한 통계를 내보겠습니다.**

In [22]:
# 유저별 몇 편의 영화를 봤는지에 대한 통계
user_count = ratings.groupby('user_id')['movie_id'].count()
user_count.describe()

count    6039.000000
mean      138.512668
std       156.241599
min         1.000000
25%        38.000000
50%        81.000000
75%       177.000000
max      1968.000000
Name: movie_id, dtype: float64

In [23]:
# 유저별 play횟수 중앙값에 대한 통계
user_median = ratings.groupby('user_id')['timestamp'].median()
user_median.describe()

count    6.039000e+03
mean     9.705019e+08
std      8.916703e+06
min      9.567058e+08
25%      9.650976e+08
50%      9.720067e+08
75%      9.748566e+08
max      1.045440e+09
Name: timestamp, dtype: float64

In [24]:
ratings.describe()

,user_id,movie_id,count,timestamp
count,836478.000000,836478.000000,836478.000000,8.364780e+05
mean,3033.120626,1849.099114,3.958293,9.721628e+08
std,1729.255651,1091.870094,0.762280,1.206216e+07
min,1.000000,1.000000,3.000000,9.567039e+08
25%,1531.000000,1029.000000,3.000000,9.652795e+08
50%,3080.000000,1747.000000,4.000000,9.728388e+08
75%,4485.000000,2763.000000,5.000000,9.752064e+08
max,6040.000000,3952.000000,5.000000,1.046455e+09


## 02. 모델 검증을 위한 데이터 추가
- 내가 선호하는 영화를 5가지 골라서 rating에 추가합니다.
    - 필요한 데이터 : user_id, movie_id, count, timestamp

In [25]:
# 내가 좋아하는 영화 5편 추가
my_fav = ['Alien (1979)' , 'Sudden Manhattan (1996)' ,'Fall (1997)' ,'Mad City (1997)' ,'Alien: Resurrection (1997)']

# my_favorite을 movie_id로 변환 
my_fav_id = [title_to_id(i) for i in my_fav]

# user_id를 'miinkang'으로 정하고, 좋아하는 영화이니 모두 5점을 부여하겠습니다. 
my_movies = pd.DataFrame({'user_id': ['miinkang']*5, 'movie_id': my_fav, 'count':[5]*5, 'timestamp':[972162800]*5})

if not ratings.isin({'user_id':['miinkang']})['user_id'].any():  # user_id에 동일 id가 있는지 확인 후 추가
    ratings = ratings.append(my_movies)                            

ratings.tail(10)       # append는 맨 뒤에 붙인다. 

,user_id,movie_id,count,timestamp,title,genre
1000203,5556.0,2198,3.0,959445515.0,Modulations (1998),Documentary
1000204,5949.0,2198,5.0,958846401.0,Modulations (1998),Documentary
1000205,5675.0,2703,3.0,976029116.0,Broken Vessels (1998),Drama
1000207,5851.0,3607,5.0,957756608.0,One Little Indian (1973),Comedy|Drama|Western
1000208,5938.0,2909,4.0,957273353.0,"Five Wives, Three Secretaries and Me (1998)",Documentary
0,miinkang,Alien (1979),5.0,972162800.0,NaN,NaN
1,miinkang,Sudden Manhattan (1996),5.0,972162800.0,NaN,NaN
2,miinkang,Fall (1997),5.0,972162800.0,NaN,NaN
3,miinkang,Mad City (1997),5.0,972162800.0,NaN,NaN
4,miinkang,Alien: Resurrection (1997),5.0,972162800.0,NaN,NaN


In [26]:
# 고유한 유저, 아티스트를 찾아내는 코드
user_unique = ratings['user_id'].unique()
movie_unique = ratings['movie_id'].unique()

# 유저, 아티스트 indexing 하는 코드 idx는 index의 약자입니다.
user_to_idx = {v:k for k,v in enumerate(user_unique)}
movie_to_idx = {v:k for k,v in enumerate(movie_unique)}

In [27]:
user_to_idx['miinkang']

6039

- 위 방법으로 바꾸려고 했지만, id중복이 되기 때문에 빠진 값으로 바꿔준다. 

In [26]:
for i in range(1, 6040):
    if i in ratings['user_id'].values:
        continue
    else:
        print(i)

3598


In [28]:
temp_user_data = ratings['user_id'].map(user_to_idx.get).dropna()
if len(temp_user_data) == len(ratings):   # 모든 row가 정상적으로 인덱싱되었다면
    print('user_id column indexing OK!!')
    ratings['user_id'] = temp_user_data   # data['user_id']을 인덱싱된 Series로 교체해 줍니다. 
else:
    print('user_id column indexing Fail!!')

# artist_to_idx을 통해 artist 컬럼도 동일한 방식으로 인덱싱해 줍니다. 
temp_movie_data = ratings['movie_id'].map(movie_to_idx.get).dropna()
if len(temp_movie_data) == len(ratings):
    print('movie id column indexing OK!!')
    ratings['movie_id'] = temp_movie_data
else:
    print('movie id column indexing Fail!!')

ratings

user_id column indexing OK!!
movie id column indexing OK!!


,user_id,movie_id,count,timestamp,title,genre
0,0,0,5.0,978300760.0,One Flew Over the Cuckoo's Nest (1975),Drama
1,1,0,5.0,978298413.0,One Flew Over the Cuckoo's Nest (1975),Drama
2,2,0,4.0,978220179.0,One Flew Over the Cuckoo's Nest (1975),Drama
3,3,0,4.0,978199279.0,One Flew Over the Cuckoo's Nest (1975),Drama
4,4,0,5.0,978158471.0,One Flew Over the Cuckoo's Nest (1975),Drama
...,...,...,...,...,...,...
0,6039,3628,5.0,972162800.0,NaN,NaN
1,6039,3629,5.0,972162800.0,NaN,NaN
2,6039,3630,5.0,972162800.0,NaN,NaN
3,6039,3631,5.0,972162800.0,NaN,NaN


## 03. CSR matrix 만들기 

In [29]:
from scipy.sparse import csr_matrix

num_user = ratings['user_id'].nunique()
num_movie = ratings['movie_id'].nunique()

csr_data = csr_matrix((ratings['count'], (ratings['user_id'], ratings['movie_id'])), shape=(num_user, num_movie))
csr_data

<6040x3633 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Row format>

## 04. 모델 학습 
- als_model = AlternatingLeastSquares 모델을 직접 구성하여 학습하기

In [30]:
from implicit.als import AlternatingLeastSquares
import os
import numpy as np

# implicit 라이브러리에서 권장하는 환경설정
os.environ['OPENBLAS_NUM_THREADS']='1'
os.environ['KMP_DUPLICATE_LIB_OK']='True'
os.environ['MKL_NUM_THREADS']='1'

In [31]:
# Implicit AlternatingLeastSquares 모델 선언
als_model = AlternatingLeastSquares(factors=3000, regularization=0.01, use_gpu=False, iterations=50, dtype=np.float32)

In [32]:
# als 모델은 input으로 (item X user 꼴의 matrix를 받기 때문에 Transpose해줍니다.)
csr_data_transpose = csr_data.T
csr_data_transpose

<3633x6040 sparse matrix of type '<class 'numpy.float64'>'
	with 836483 stored elements in Compressed Sparse Column format>

In [33]:
# 모델 훈련
als_model.fit(csr_data_transpose)

  0%|          | 0/50 [00:00<?, ?it/s]

## 05. 선호하는 영화를 이용해 학습된 모델이 예측한 선호도를 확인하기

In [81]:
miinkang, alien = user_to_idx['miinkang'], movie_to_idx['Alien: Resurrection (1997)']
miinkang_vector, alien_vector = als_model.user_factors[miinkang], als_model.item_factors[alien]

In [36]:
miinkang_vector

array([ 0.42108274, -0.1434877 , -0.10709431, ..., -0.0661026 ,
        0.1046145 ,  0.03232726], dtype=float32)

In [37]:
alien_vector

array([0.00748233, 0.00262783, 0.00422243, ..., 0.00409044, 0.00725666,
       0.00625757], dtype=float32)

- 두 벡터를 곱해 1에 가까운 수가 나올 수록 학습이 잘 된 것이다.

In [38]:
np.dot(miinkang_vector, alien_vector)

0.9511499

- 너무 낮기 때문에, iteration, factors를 높여보겠다.
    - 0.00004xxx -> 0.0193257 로 높였다. 
- factors 100-> 500
    - 0.0193257 -> 0.04424449
- iteration 500 -> 1000
    - 0.04424449 -> 8.422308e-10
- iteration 50, fators 500 -> 3000
    - 8.422308e-10 -> 0.9511499

**iteration보다 fators를 늘이는 것이 학습이 잘 되었다.**

### 선호하는 영화 리스트에 없었던 영화와 비교해보겠습니다.

In [56]:
# 선호하는 영화들의 장르 확인하기 

for i in my_fav:
    print(ratings[ratings['title']==i]['genre'].unique())

['Action|Horror|Sci-Fi|Thriller']
[]
['Romance']
['Action|Drama']
['Action|Horror|Sci-Fi']


In [100]:
ratings[ratings['genre']=='Action|Horror|Sci-Fi']['title'].unique()

array(['Alien: Resurrection (1997)', 'Hidden, The (1987)',
       'Night of the Comet (1984)', 'Deep Rising (1998)',
       'Hellraiser: Bloodline (1996)', 'Alligator (1980)',
       'Chopping Mall (a.k.a. Killbots) (1986)'], dtype=object)

In [101]:
title_to_id('Deep Rising (1998)')

1762

In [107]:
# deep = movie_to_idx['Deep Rising (1998)']  # 왜인지 안된다.. 
deep_vector = als_model.item_factors[1762]
np.dot(miinkang_vector, deep_vector)

0.010117743

낮게 나왔다..
## 06. 내가 좋아하는 영화와 비슷한 영화를 추천받기
- AlternatingLeastSquares의 similar_items 메서드를 통해 비슷한 영화를 찾기

In [105]:
favorite_movie = 'Alien (1979)'
movie_id = movie_to_idx[favorite_movie]
similar_artist = als_model.similar_items(movie_id, N=15)
similar_artist

[(3628, 1.0000001),
 (3632, 0.89860016),
 (3631, 0.89265996),
 (3630, 0.8847001),
 (3629, 0.8752242),
 (3587, 0.86790055),
 (3579, 0.867627),
 (3593, 0.86609346),
 (3478, 0.86546844),
 (3327, 0.8644257),
 (3598, 0.8639781),
 (3594, 0.86375123),
 (3613, 0.8624202),
 (3484, 0.86116225),
 (3459, 0.8602585)]

In [115]:
# (movie_id, 유사도)
# movie_id를 tilte로 변환합니다.
for i in [id_to_title(i[0]) for i in similar_artist]:
    print(ratings[ratings['title']==i]['genre'].unique())
[id_to_title(i[0]) for i in similar_artist]

['Action|Drama|War']
['Comedy']
['Comedy']
[]
['Comedy']
['Horror']
['Drama']
['Action|Sci-Fi']
['Drama']
['Documentary']
['Drama']
['Drama']
['Comedy']
['Thriller']
['Drama|Horror']


['Flying Tigers (1942)',
 'Monsieur Verdoux (1947)',
 "It's in the Water (1998)",
 "House of Exorcism, The (La Casa dell'esorcismo) (1974)",
 'Gold Rush, The (1925)',
 'Inferno (1980)',
 'I Dreamed of Africa (2000)',
 'Battlefield Earth (2000)',
 'Bamba, La (1987)',
 'Beyond the Mat (2000)',
 'Hamlet (2000)',
 'Center Stage (2000)',
 'Things Change (1988)',
 'Skulls, The (2000)',
 'Gothic (1986)']

**장르를 확인해보니 3가지만 좋아하는 영화의 장르와 같은 장르이다.**
- 함수로 만들기

In [124]:
def get_similar_movie(movie_name: str):
    movie_name = movie_name
    movie_id = movie_to_idx[movie_name]
    similar_movie = als_model.similar_items(movie_id)
    similar_movie = [id_to_title(i[0]) for i in similar_movie]
    return similar_movie

## 07. 내가 가장 좋아할 만한 영화들을 추천받아 봅시다.
- AlternatingLeastSquares의 recommend 메서드를 통해 비슷한 영화를 찾기

In [126]:
user = user_to_idx['miinkang']
# recommend에서는 user*item CSR Matrix를 받습니다.
movie_recommended = als_model.recommend(user, csr_data, N=20, filter_already_liked_items=True)
movie_recommended

[(3543, 0.08742896),
 (3602, 0.08175069),
 (2910, 0.07100366),
 (3520, 0.066416845),
 (3489, 0.06490412),
 (3510, 0.059789747),
 (2682, 0.05894955),
 (2183, 0.057301566),
 (3593, 0.056624465),
 (3394, 0.054914333),
 (2676, 0.05396311),
 (3407, 0.05388003),
 (2790, 0.052828714),
 (3395, 0.052022915),
 (2990, 0.051221434),
 (3160, 0.05000774),
 (3555, 0.048234966),
 (3187, 0.047321834),
 (2802, 0.046658874),
 (3286, 0.046518803)]

In [130]:
[id_to_title(i[0]) for i in movie_recommended]

['Diner (1982)',
 'G. I. Blues (1960)',
 "Ennui, L' (1998)",
 'How to Stuff a Wild Bikini (1965)',
 'Hook (1991)',
 'Frequency (2000)',
 'Limbo (1999)',
 'Man Who Knew Too Much, The (1956)',
 'Battlefield Earth (2000)',
 'Blind Date (1987)',
 'Instinct (1999)',
 'Carriers Are Waiting, The (Les Convoyeurs Attendent) (1999)',
 'Final Conflict, The (a.k.a. Omen III: The Final Conflict) (1981)',
 'Nadine (1987)',
 'Licence to Kill (1989)',
 'Magnolia (1999)',
 'U-571 (2000)',
 'Trans (1998)',
 'Tequila Sunrise (1988)',
 'Snow Day (2000)']

In [131]:
for i in [id_to_title(i[0]) for i in movie_recommended]:
    print(ratings[ratings['title']==i]['genre'].unique())

['Comedy|Drama']
['Musical']
[]
['Comedy']
['Adventure|Fantasy']
['Drama|Thriller']
['Drama']
['Thriller']
['Action|Sci-Fi']
['Comedy|Romance']
['Drama|Thriller']
['Comedy|Drama']
['Horror']
['Comedy']
['Action']
['Drama']
['Action|Thriller']
['Drama']
['Action|Romance|Thriller']
['Comedy']


**뮤지컬, 코미디를 제외하고 다 좋아하는 영화의 장르가 나왔다.**
- 이제  AlternatingLeastSquares 클래스에 구현된 explain 메소드를 이용해 추천시스템에 기여한 정도를 확인합니다.

In [132]:
alien = movie_to_idx['Alien (1979)']
explain = als_model.explain(user, csr_data, itemid=alien)

In [133]:
explain

(0.9002295347956508,
 [(3628, 0.6565555105936658),
  (3631, 0.07160090897228938),
  (3632, 0.06873689239842334),
  (3630, 0.0632145828176548),
  (3629, 0.04012164001361762)],
 (array([[ 0.48148632,  0.19070034,  0.17290259, ...,  0.19827063,
           0.1965718 ,  0.19098492],
         [ 0.09181961,  0.43556172,  0.1130279 , ...,  0.12026023,
           0.12590529,  0.12677351],
         [ 0.08325023,  0.08220321,  0.41203133, ...,  0.08798368,
           0.08409664,  0.08851522],
         ...,
         [ 0.09546459,  0.09019103,  0.0841263 , ...,  0.28165356,
          -0.00548367,  0.00183927],
         [ 0.09464663,  0.09232583,  0.08286904, ...,  0.09446497,
           0.28534116,  0.00507954],
         [ 0.09195663,  0.09163858,  0.08382177, ...,  0.09315241,
           0.09770541,  0.28085487]]),
  False))

**explain에 (영화, 기여도) 가 나와있다.**

In [134]:
[(id_to_title(i[0]), i[1]) for i in explain[1]]

[('Flying Tigers (1942)', 0.6565555105936658),
 ("It's in the Water (1998)", 0.07160090897228938),
 ('Monsieur Verdoux (1947)', 0.06873689239842334),
 ("House of Exorcism, The (La Casa dell'esorcismo) (1974)",
  0.0632145828176548),
 ('Gold Rush, The (1925)', 0.04012164001361762)]

- 가장 높은 기여도를 가진 'Flying Tigers (1942)'영화를 확인해보니, 장르가 선호하는 영화와 유사하다.
- 가장 낮은 기여도를 가진 'Gold Rush, The (1925)'는 코미디 영화이다. (선호하는 영화 중 없었음)

# 정리     
---        
1. 데이터 준비와 전처리
    - ratings에 있는 유니크한 영화 개수
    - rating에 있는 유니크한 사용자 수
    - 가장 인기 있는 영화 30개(인기순)
    - meta data(movies)와 ratings를 합친다.
    - 평점 3점 이하의 데이터는 삭제한다.
2. 내가 선호하는 영화를 5가지 골라서 rating에 추가
    - 이를 통해서 후에 학습한 모델을 평가한다.
3. CSR matrix 생성
4. als_model = AlternatingLeastSquares 모델 구성 후 학습
5. 모델 평가
    - 추가한 데이터를 통해 선호도 추출
    - 비슷한 영화 추출
    - 사용자에게 추천하기


# 루브릭 평가
---
1. **CSR matrix가 정상적으로 만들어졌다.**   
    - 잘 만들어졌음을 확인할 수 있다.


2. **MF 모델이 정상적으로 훈련되어 그럴듯한 추천이 이루어졌다.**  
    - 기존 추가한 영화와의 선호도는 잘 나왔다.
    
     
3. **비슷한 영화 찾기와 유저에게 추천하기의 과정이 정상적으로 진행되었다.**  
    - 영화간 유사도는 의미있지 않았지만, 기여도는 의미있게 측정이 되었다. 

    
# 회고
---
## 느낀 점
- 영화 데이터 중 아는 영화가 거의 없어서 장르를 기준으로 판단할 수 밖에 없었던 점이 아쉬웠다.
- MF 매트릭스, ALS 모델이 어떻게 구성되는지 공부할 수 있었다.
- 하지만 단지 선호 여부에 따라서만 판단하는 점이 아쉽다.
- 보다 세밀하게 구성되어있는 모델을 공부해보고 싶다. 